# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [33]:
# import 
import os 
import requests 
import json
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama
import google.generativeai as genai
import gradio as gr
import time
import base64
from io import BytesIO
from pydub import AudioSegment
from pydub.playback import play

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
MODEL_GEMINI = 'gemini-1.5-flash'

In [3]:
# set up environment

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else: 
    print("OpenAI API Key not set")
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AIzaSyAG


In [4]:
# connect to LLM Models

openai = OpenAI()
google.generativeai.configure()

In [5]:
# system_prompts

system_prompt = "You are a code assistant helping a beginner software development student."
system_prompt += "Your explanations should be clear, structured, and beginner-friendly."
system_prompt += "If user ask any unrelevant questions, you will say you don't know and you can only answer to coding questions."

In [ ]:
# test function chat

def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]

    # stream OpenAI 
    stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
# gradio test simple UI

gr.ChatInterface(fn=stream_gemini, type="messages").launch()

In [27]:
# stream model gpt

def stream_gpt(prompt, history=[]):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": prompt}]

    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        stream=True
    )
    
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

    talker(result)

In [30]:
# stream model llama

def stream_llama(prompt, history=[]):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": prompt}]

    stream = ollama.chat(
        model=MODEL_LLAMA,
        messages=messages,
        stream=True
    )

    result = ""
    for chunk in stream:
        result += chunk["message"].get("content", "")
        yield result 

    talker(result)

In [36]:
# stream model gemini

def transform_history(history):
    new_history = []
    for chat in history:
        new_history.append({"parts": [{"text": chat[0]}], "role": "user"})
        new_history.append({"parts": [{"text": chat[1]}], "role": "model"})
    return new_history

def stream_gemini(prompt, history=[]):
    model = genai.GenerativeModel(
        model_name=MODEL_GEMINI,
        system_instruction=system_prompt,
        safety_settings=[
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"}
        ]
    )

    chat = model.start_chat(history=transform_history(history))  # Set up chat with history

    response = chat.send_message(prompt)
    response.resolve()  # Ensure we get the full response before streaming

    # Each character of the answer is displayed
    for i in range(len(response.text)):
        time.sleep(0.01)
        yield response.text[: i+1]



Input #0, wav, from '/var/folders/1q/nyt9ydwd0j72jcxlq0jbp8300000gn/T/tmpk9b3hnar.wav':
  Duration: 00:00:06.98, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   6.88 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 

In [9]:
# choose llm model

def stream_llm_model(prompt, history, model):
    if model == "GPT":
        result = stream_gpt(prompt, history=[])
    elif model == "Gemini":
        result = stream_gemini(prompt, history=[])
    elif model == "Llama":
        result = stream_llama(prompt, history=[])
    else:
        raise ValueError("Unknown model")
    yield from result

In [16]:
# add audio

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message
    )

    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [17]:
# test audio func

talker("Hello, I am an AI assistant.")

Input #0, wav, from '/var/folders/1q/nyt9ydwd0j72jcxlq0jbp8300000gn/T/tmpjmp8h1x1.wav':
  Duration: 00:00:01.70, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   1.47 M-A: -0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 

   1.58 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 

In [38]:
# Gradio UI

chat = gr.ChatInterface(
    fn=stream_llm_model,
    additional_inputs=[
        gr.Dropdown(["GPT", "Gemini", "Llama"], label="Select model")
    ],
    title="AI Models Coding Assistant",
    description="Select a model and start chatting about coding question.",
    type="messages"
)

chat.launch(share=True)

* Running on local URL:  http://127.0.0.1:7869
* Running on public URL: https://b28107a9d90f5dfc81.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
